In [1]:
import sys
sys.path.insert(0,"/mnt/c/gitreps/SlowQuant/")
import numpy as np
import time
import matplotlib.pyplot as plt
from slowquant import shortcuts as SQ

In [2]:
molecule = np.array([[4,0,0,0],
                [4,0.0,0.0,0.0]])

In [3]:
# Uncopuled perturbed Hartree-Fock from:
# Introduction to the calculation of molecular properties by response theory
# by Julien Toulouse

# Implemented in MO spin orbital basis, might not need spin orbital basis?
# Not tested, so not sure if working
def UCHF_polarizability(occ, F, C, mu_x, mu_y, mu_z):
    Fspin = np.zeros((len(F)*2,len(F)*2))
    Cspin = np.zeros((len(F)*2,len(F)*2))
    for p in range(1,len(F)*2+1):
        for q in range(1,len(F)*2+1):
            Fspin[p-1,q-1] = F[(p+1)//2-1,(q+1)//2-1] * (p%2 == q%2)
            Cspin[p-1,q-1] = C[(p+1)//2-1,(q+1)//2-1] * (p%2 == q%2)
    FMOspin = np.dot(np.transpose(Cspin),np.dot(Fspin,Cspin))
    
    alpha = np.zeros((3,3))
    mu_tot = np.zeros((len(mu_x),len(mu_x),3))
    mu_tot[:,:,0] = mu_x
    mu_tot[:,:,1] = mu_y
    mu_tot[:,:,2] = mu_z
    # x = 0, y = 1, z = 2
    for i in range(0, 3):
        for j in range(0, 3):
            for a in range(0, occ):
                for r in range(occ, len(F)*2):
                    alpha[i,j] += (mu_tot[r,a,i]*mu_tot[a,r,j])/(FMOspin[r,r]-FMOspin[a,a])
    
    alpha = 2.0*alpha
    print(alpha)

def Transform_dipole_MOspin(mu_x, mu_y, mu_z, C):
    mu_xMO = np.zeros((len(mu_x),len(mu_x)))
    mu_yMO = np.zeros((len(mu_x),len(mu_x)))
    mu_zMO = np.zeros((len(mu_x),len(mu_x)))
    for p in range(0, len(C)):
        for q in range(0, len(C)):
            for mu in range(0, len(C)):
                for nu in range(0, len(C)):
                    mu_xMO[p,q] += C[mu,p]*C[nu,q]*mu_x[mu,nu]
                    mu_yMO[p,q] += C[mu,p]*C[nu,q]*mu_y[mu,nu]
                    mu_zMO[p,q] += C[mu,p]*C[nu,q]*mu_z[mu,nu]
            
    mu_xspin = np.zeros((len(mu_x)*2,len(mu_x)*2))
    mu_yspin = np.zeros((len(mu_x)*2,len(mu_x)*2))
    mu_zspin = np.zeros((len(mu_x)*2,len(mu_x)*2))
    for p in range(1,len(mu_x)*2+1):
        for q in range(1,len(mu_x)*2+1):
            mu_xspin[p-1,q-1] = mu_xMO[(p+1)//2-1,(q+1)//2-1] * (p%2 == q%2)
            mu_yspin[p-1,q-1] = mu_yMO[(p+1)//2-1,(q+1)//2-1] * (p%2 == q%2)
            mu_zspin[p-1,q-1] = mu_zMO[(p+1)//2-1,(q+1)//2-1] * (p%2 == q%2)
    
    return mu_xspin, mu_yspin, mu_zspin


In [4]:
VNN, VNe, S, Te, Vee = SQ.Integrals(molecule, "3-21G")
EHF, C, F, D = SQ.HartreeFock(molecule, VNN, Te, S, VNe, Vee)
mu_x, mu_y, mu_z = SQ.dipoleintegral(molecule, "3-21G")
mu_xspin, mu_yspin, mu_zspin = Transform_dipole_MOspin(mu_x, mu_y, mu_z, C)
occ = int(molecule[0,0])

Iter	 Eel	
1 	 -14.1747842353 	 -14.1747842353 	  5.25012207e+00 	  9.44202705e-01
2 	 -14.5005469173 	 -14.5005469173 	 -3.25762682e-01 	  2.45249616e-01 	  1.45640345e-01
3 	 -14.4866262751 	 -14.4866262751 	  1.39206422e-02 	  4.69794922e-03 	  1.45640345e-01
4 	 -14.4868217439 	 -14.4868217439 	 -1.95468797e-04 	  4.12418596e-05 	  1.45640345e-01
5 	 -14.4868202417 	 -14.4868202417 	  1.50218177e-06 	  3.42360202e-07 	  1.45640345e-01
6 	 -14.4868202419 	 -14.4868202419 	 -1.82698301e-10 	  7.15068061e-11 	  1.45640345e-01


In [5]:
UCHF_polarizability(occ, F, C, mu_xspin, mu_yspin, mu_zspin)

[[  2.85757090e+01   1.87350135e-16  -1.87948447e-15]
 [  2.84494650e-16   2.85757090e+01  -1.29015904e-15]
 [ -1.87948447e-15  -8.46069827e-16   2.85757090e+01]]
